# Google form analysis

Analysis of results extracted from Google forms in csv format.

## Preparation

In [1]:
%run "../Utilities/Preparation.ipynb"
%run "../Static data/Google form correct answers.ipynb"

/Users/raphik12/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/raphik12/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Constants

In [2]:
gformPath = "../../data/Google forms/"
genericFormName = '2017-11-02-gform-'
enSuffix = 'en'
frSuffix = 'fr'
csvSuffix = '.csv'
gformEN = pd.read_csv(gformPath + genericFormName + enSuffix + csvSuffix)
gformFR = pd.read_csv(gformPath + genericFormName + frSuffix + csvSuffix)

### en-fr gform merge

In [3]:
gformEN['Language'] = pd.Series('en', index=gformEN.index)
gformFR['Language'] = pd.Series('fr', index=gformFR.index)
gformFR.columns = gformEN.columns
gform = pd.concat([gformEN, gformFR])

In [4]:
gform.head(2)

,Timestamp,Are you interested in video games?,Do you play video games?,How old are you?,What is your gender?,How long have you studied biology?,Are you interested in biology?,"Before playing Hero.Coli, had you ever heard about synthetic biology?","Before playing Hero.Coli, had you ever heard about BioBricks?",Have you ever played an older version of Hero.Coli before?,...,What does this device do?.6,What does this device do?.7,"Guess: what would a device producing l-arabinose do, if it started with a l-arabinose-induced promoter?",Guess: the bacterium would glow yellow...,What is the species of the bacterium of the game?,What is the scientific name of the tails of the bacterium?,Find the antibiotic:,You can write down remarks here.,Do not edit - pre-filled anonymous ID,Language
0,2017/08/18 3:22:26 PM GMT+1,Extremely,Extremely,23,Female,Until bachelor's degree,Extremely,Yes,NaN,No,...,It generates antibiotic resistance,It generates green fluorescence in presence of...,"After being induced, it would produce more and...",I don't know,E. Coli,Flagella,Ampicillin,Your game is amazing. One suggestion. In sever...,8d352896-a3f1-471c-8439-0f426df901c1,en
1,2017/09/06 1:28:03 PM GMT+1,Moderately,Moderately,28,Other,Until the end of high school,Moderately,Yes,NaN,No,...,It generates antibiotic resistance,It generates green fluorescence in presence of...,"After being induced, it would produce more and...",I don't know,E. Coli,Flagella,Ampicillin,"This was a nice educational game, thank you fo...",7037c5b2-c286-498e-9784-9a061c778609,en


In [5]:
gform.tail(2)

,Timestamp,Are you interested in video games?,Do you play video games?,How old are you?,What is your gender?,How long have you studied biology?,Are you interested in biology?,"Before playing Hero.Coli, had you ever heard about synthetic biology?","Before playing Hero.Coli, had you ever heard about BioBricks?",Have you ever played an older version of Hero.Coli before?,...,What does this device do?.6,What does this device do?.7,"Guess: what would a device producing l-arabinose do, if it started with a l-arabinose-induced promoter?",Guess: the bacterium would glow yellow...,What is the species of the bacterium of the game?,What is the scientific name of the tails of the bacterium?,Find the antibiotic:,You can write down remarks here.,Do not edit - pre-filled anonymous ID,Language
38,2017/10/26 4:27:48 PM GMT+1,Beaucoup,Enormément,19,Homme,Jusqu'à la license,Beaucoup,Oui,Oui,Non,...,Elle permet de résister aux antibiotiques,Elle produit la fluorescence verte en présence...,Elle ne serait active que dans les nuages de l...,Si elle produisait de la YFP sous lumière cyan,E. Coli,Des flagelles,Ampicilline,"Problèmes de performances ( fps drop ), sinon ...",74cfbbd0-eb35-467c-9bad-221ac73a7444,fr
39,2017/10/28 2:32:13 PM GMT+1,Moyennement,Moyennement,11,Homme,"Jamais, pas même au collège",Un peu,Non,Non,Non,...,Elle permet de résister aux antibiotiques,Elle produit la fluorescence verte en présence...,Je ne sais pas,Je ne sais pas,E. Coli,Des flagelles,Je ne sais pas,NaN,bdc1c38d-dce7-4c00-8ea2-9514643bc09f,fr


In [6]:
#localplayerguidkey = 'Ne pas modifier - identifiant anonyme prérempli'
localplayerguidkey = 'Do not edit -  pre-filled anonymous ID'
localplayerguidindex = gform.columns.get_loc(localplayerguidkey)
localplayerguidindex

41

In [7]:
firstEvaluationQuestionKey = 'In order to modify the abilities of the bacterium, you have to...'
firstEvaluationQuestionIndex = gform.columns.get_loc(firstEvaluationQuestionKey)
firstEvaluationQuestionIndex

13

In [ ]:
answersColumnNameStem = "answers"
correctionsColumnNameStem = "corrections"

# Functions
<a id=functions />

In [4]:
#user id in format localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'

def getAllResponders():
    userIds = gform[localplayerguidkey].values
    result = []
    for id in userIds:
        result.append('"' + id + '"')
    return result

def hasAnswered( userId ):
    return userId.replace('"','') in gform[localplayerguidkey].values

def getAnswers( userId ):
    answers = gform[gform[localplayerguidkey]==userId.replace('"','')]
    _columnAnswers = answers.T
    
    if 0 != len(answers):
        _newColumns = []
        for column in _columnAnswers.columns:
            _newColumns.append(answersColumnNameStem + str(column))
        _columnAnswers.columns = _newColumns
    else:
        # user has never answered
        print("user " + str(userId) + " has never answered")
        
    return _columnAnswers

def getCorrections( userId ):
    _columnAnswers = getAnswers( userId )
    
    if 0 != len(_columnAnswers.columns):

        _questionsCount = len(columnAnswers.values)

        for _columnName in _columnAnswers.columns:
            if answersColumnNameStem in _columnName:
                _answerNumber = _columnName.replace(answersColumnNameStem,"")
                _newCorrectionsColumnName = correctionsColumnNameStem + _answerNumber
                _columnAnswers[_newCorrectionsColumnName] = _columnAnswers[_columnName]

                for _index in range(0, firstEvaluationQuestionIndex):
                    _columnAnswers[_newCorrectionsColumnName][_index] = 'NaN'
                for _index in range(firstEvaluationQuestionIndex, len(_columnAnswers.values)-3):
                    _columnAnswers[_newCorrectionsColumnName][_index] =  \
                        str(columnAnswers[columnName][index]).startswith(str(correctAnswers[columnAnswers[columnName]['Language']][_index]))
                        
                _columnAnswers[_newCorrectionsColumnName][_questionsCount-3] = 'NaN'
                _columnAnswers[_newCorrectionsColumnName][_questionsCount-2] = 'NaN'
                _columnAnswers[_newCorrectionsColumnName][_questionsCount-1] = 'NaN'
        return _columnAnswers
    else:
        # user has never answered
        print("can't give correct answers")
    return _columnAnswers

#function that returns the score from user id
def getScore( userId ):
    if hasAnswered( userId ):
        _columnAnswers = getCorrections( userId)
        score = (_columnAnswers['correct']).value_counts()[True]
        return score
    else:
        return np.NaN

#function that returns the list of checkpoints from user id
def getValidatedCheckpoints( userId ):
    _columnAnswers = getCorrections( userId)
    questionnaireValidatedCheckpointsPerQuestion = pd.Series(np.nan, index=range(len(checkpointQuestionMatching)))
    
    for index in range(0, len(questionnaireValidatedCheckpointsPerQuestion)):
        if _columnAnswers['correct'][index]==True:
            questionnaireValidatedCheckpointsPerQuestion[index] = checkpointQuestionMatching['checkpoint'][index]
        else:
            questionnaireValidatedCheckpointsPerQuestion[index] = ''
    
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpointsPerQuestion.unique()
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints[questionnaireValidatedCheckpoints!='']
    questionnaireValidatedCheckpoints = pd.Series(questionnaireValidatedCheckpoints)
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints.sort_values()
    questionnaireValidatedCheckpoints.index = range(0, len(questionnaireValidatedCheckpoints))
    return questionnaireValidatedCheckpoints

def getNonValidated( checkpoints ):
    return pd.Series(np.setdiff1d(checkpointQuestionMatching['checkpoint'].values, checkpoints.values))

def getNonValidatedCheckpoints( userId ):
    validated = getValidatedCheckpoints( userId )
    return getNonValidated(validated)